In [1]:
from RegistrationObj import RegistrationObj, IterativePairwise
import scipy.stats as st
from utils import colour_map, min_max_normalisation, mae, mae_permutation
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from open3d import JVisualizer
import tifffile.tifffile as tiff


# Introduction

# 18ss TCF-GFP

In [3]:
pos_path_source_list_18ss = ["./datasets/4-multiway/18ss/18ss_position_1.csv",
                  "./datasets/4-multiway/18ss/18ss_position_2.csv"]

color_path_source_list_18ss = ["./datasets/4-multiway/18ss/18ss_TCF_mRNA_1.csv",
                  "./datasets/4-multiway/18ss/18ss_TCF_mRNA_2.csv"]

pos_path_target_18ss = "./datasets/4-multiway/18ss/18ss_position_3.csv"
color_path_target_18ss = "./datasets/4-multiway/18ss/18ss_TCF_mRNA_3.csv"

In [4]:
tcf_18ss = IterativePairwise(pos_path_source_list_18ss, pos_path_target_18ss, color_path_source_list_18ss, color_path_target_18ss,
                             algorithm = "colored_icp",pos_skiprows = [0,1,2], pos_usecols = [0,1,2], color_skiprows = [0,1,2], color_usecols = [0])

tcf_18ss.iterative_registration(jupyter_visualise = True, verbose = False)

Using median averaging
Using median averaging


JVisualizer with 1 geometries

PointCloud with 1642 points.

In [5]:
tcf18ss_pcd = tcf_18ss.iterative_registration(jupyter_visualise = False, verbose = False)

Using median averaging
Using median averaging


In [6]:
import copy 
tcf18ss_copy = copy.deepcopy(tcf18ss_pcd)
intensity_array = tcf_18ss.registered_color

In [7]:
intensity_array

array([[10.94894  ],
       [13.6116   ],
       [ 4.96027  ],
       ...,
       [ 7.88553  ],
       [ 5.3544175],
       [ 6.15034  ]])

## (Dillan) Exporting image

In [296]:
export_pcd = np.asarray(tcf18ss_copy.points)
print(export_pcd)

x_res=xdim_px/(191)
y_res=ydim_px/(191)
z_res= zdim_px/(77.1571)

zdim_px = 150
xdim_px = 1000
ydim_px = 1000
spot_diameter=2


[[163.5     93.0311 254.094 ]
 [168.642   90.131  254.786 ]
 [174.297   95.4054 255.68  ]
 ...
 [168.799   75.2722 184.06  ]
 [160.292   63.7662 185.585 ]
 [176.96    71.3038 186.345 ]]


In [299]:
export_image=np.zeros([zdim_px,xdim_px,ydim_px])

export_x_location=np.round((export_pcd[:,0]-np.min(export_pcd[:,0]))*x_res).astype('int')
export_y_location=np.round((export_pcd[:,1]-np.min(export_pcd[:,1]))*y_res).astype('int')
export_z_location=np.round((export_pcd[:,2]-np.min(export_pcd[:,2]))*z_res).astype('int')

pixel_radius=int(np.round(x_res*(spot_diameter/2)))

for i in range(len(export_pcd)):
    print(len(export_pcd)-i,'remaining')
    center_point=export_x_location[i],export_y_location[i],export_z_location[i]
    print(center_point)
    for x in range(center_point[0]-pixel_radius,center_point[0]+pixel_radius):
        for y in range(center_point[1]-pixel_radius, center_point[1]+pixel_radius):
            for z in range(center_point[2]-pixel_radius,center_point[2]+pixel_radius):
                length=np.sqrt((x-center_point[0])**2+(y-center_point[1])**2+(z-center_point[2])**2)

            if length<=int(np.round(pixel_radius)):
                export_image[z,x,y]=intensity_array[i]

#tiff.imwrite("my_test_image_2.tif",export_image.astype("uint8"), compression='jpeg')
#print('Saved',image_name,'as Tiff')

1642 remaining
(258, 321, 150)


IndexError: index 150 is out of bounds for axis 0 with size 150

## (Kane) Exporting Image

First, given an arbitrary range of x, y, z coordinate values, we scale them to fit into an image of dimension `n_depth` x `n_width` x `n_height` x `n_colors`.

### Pcd and colors

In [9]:
export_pcd = np.asarray(tcf18ss_copy.points)
intensity_array = tcf_18ss.registered_color

### Setting parameters

In [10]:
width = 1024
height = 1024
depth = 110
color = 255

### Interpolation

In [17]:
from scipy.interpolate import interp1d

x_lim = [np.min(export_pcd[:,0]), np.max(export_pcd[:,0]) ]
y_lim = [np.min(export_pcd[:,1]), np.max(export_pcd[:,1]) ]
z_lim = [np.min(export_pcd[:,2]), np.max(export_pcd[:,2])]
color_lim = [np.min(intensity_array), np.max(intensity_array)]

x_interp = interp1d(x_lim, [0,width-1])
y_interp = interp1d(y_lim, [0,height-1])
z_interp = interp1d(z_lim, [0,depth-1])
color_interp = interp1d(color_lim, [0,color])

data_x_interp = np.round(x_interp(export_pcd[:,0]))
data_y_interp = np.round(y_interp(export_pcd[:,1]))
data_z_interp = np.round(z_interp(export_pcd[:,2]))
data_col_interp = color_interp(intensity_array).squeeze()

In [12]:
combined_zyxc = np.vstack([data_z_interp, data_x_interp, data_y_interp, data_col_interp]).astype('int')

In [14]:
export_image=np.zeros([depth, width, height])

### Reconstructing image

In [15]:
for i in range(combined_zyxc.shape[1]):
    export_image[combined_zyxc[0][i], combined_zyxc[1][i], combined_zyxc[2][i]] = combined_zyxc[3][i]

    

### Morphological operations

In [22]:
from skimage.morphology import erosion, dilation, opening, closing, white_tophat

dilated = dilation(export_image)
closing = closing(export_image)
tiff.imwrite("closing.tif",closing.astype("uint8"), compression='jpeg')

### Saving image

In [456]:
tiff.imwrite("kane_test.tif",export_image.astype("uint8"), compression='jpeg')

In [459]:
export_image[100]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Results

In [4]:
tcf_18ss
#print(tcf_18ss.combined_results)
#print(tcf_18ss.registered_color)

--- Iterative Pairwise Object--- 
Algorithm used = colored_icp
Source file names : ['./datasets/4-multiway/18ss/18ss_position_1.csv', './datasets/4-multiway/18ss/18ss_position_2.csv']
Number of source files : 2
Target file name: ./datasets/4-multiway/18ss/18ss_position_3.csv

In [5]:
tcf_18ss.source_pcd

[PointCloud with 1784 points., PointCloud with 1689 points.]

# 21ss TCF-GFP

In [6]:
pos_path_source_list_21ss = ["./datasets/4-multiway/21ss/21ss_position_2_Detailed.csv", 
                            "./datasets/4-multiway/21ss/21ss_position_3_Detailed.csv", 
                            "./datasets/4-multiway/21ss/21ss_position_4_Detailed.csv", 
                            "./datasets/4-multiway/21ss/21ss_position_5_Detailed.csv", 
                            "./datasets/4-multiway/21ss/21ss_position_6_Detailed.csv"]

color_path_source_list_21ss = ["./datasets/4-multiway/21ss/21ss_TCF_mRNA_2_Detailed.csv", 
                            "./datasets/4-multiway/21ss/21ss_TCF_mRNA_3_Detailed.csv", 
                            "./datasets/4-multiway/21ss/21ss_TCF_mRNA_4_Detailed.csv", 
                            "./datasets/4-multiway/21ss/21ss_TCF_mRNA_5_Detailed.csv", 
                            "./datasets/4-multiway/21ss/21ss_TCF_mRNA_6_Detailed.csv"]

pos_path_target_21ss = "./datasets/4-multiway/21ss/21ss_position_1_Detailed.csv"
color_path_target_21ss = "./datasets/4-multiway/21ss/21ss_TCF_mRNA_1_Detailed.csv"

In [7]:
tcf_21ss = IterativePairwise(pos_path_source_list_21ss, pos_path_target_21ss, color_path_source_list_21ss, color_path_target_21ss,
                             algorithm = "colored_icp",pos_skiprows = [0,1,2], pos_usecols = [0,1,2], color_skiprows = [0,1,2], color_usecols = [0])

tcf_21ss.iterative_registration(jupyter_visualise = True, verbose = False)

Using median averaging
Using median averaging
Using median averaging
Using median averaging
Using median averaging


JVisualizer with 1 geometries

PointCloud with 2445 points.

In [19]:
visualizer = JVisualizer()
visualizer.add_geometry(tcf_21ss.source_pcd[0])
visualizer.show()

JVisualizer with 1 geometries

## Results

In [8]:
tcf_21ss
print(tcf_21ss.combined_results)
print(tcf_21ss.registered_color)

[[0.9934043632673769, 3.9496800635776044, 3.975903679934351, 13.92587076746204], [0.9647058823529412, 4.003149887730837, 4.149606590940501, 15.668203562498281], [1.0, 3.6675440824630114, 3.6675440824630114, 10.908370207223198], [0.9782383419689119, 4.1373649807313715, 4.229403820345099, 13.697322239855776], [0.9861954721148537, 4.262351243539903, 4.322014614810058, 8.661850165514954]]
[[23.6356]
 [21.0235]
 [16.7638]
 ...
 [13.0945]
 [11.3764]
 [13.4074]]


# 24ss TCF-GFP

In [9]:
pos_path_source_list_24ss = ["./datasets/4-multiway/24ss/24ss_positions_2_Detailed.csv", 
                            "./datasets/4-multiway/24ss/24ss_positions_4_Detailed.csv", 
                            "./datasets/4-multiway/24ss/24ss_positions_5_Detailed.csv"]

color_path_source_list_24ss = ["./datasets/4-multiway/24ss/24ss_TCF_mRNA_2_Detailed.csv", 
                            "./datasets/4-multiway/24ss/24ss_TCF_mRNA_4_Detailed.csv", 
                            "./datasets/4-multiway/24ss/24ss_TCF_mRNA_5_Detailed.csv"]

pos_path_target_24ss = "./datasets/4-multiway/24ss/24ss_positions_1_Detailed.csv"
color_path_target_24ss = "./datasets/4-multiway/24ss/24ss_TCF_mRNA_1_Detailed.csv"

In [10]:
tcf_24ss = IterativePairwise(pos_path_source_list_24ss, pos_path_target_24ss, color_path_source_list_24ss, color_path_target_24ss,
                             algorithm = "colored_icp",pos_skiprows = [0,1,2], pos_usecols = [0,1,2], color_skiprows = [0,1,2], color_usecols = [0])

tcf_24ss.iterative_registration(jupyter_visualise = True, verbose = False)

Using median averaging
Using median averaging
Using median averaging


JVisualizer with 1 geometries

PointCloud with 1133 points.

## Results

In [11]:
print(tcf_24ss)
print(tcf_24ss.combined_results)
print(tcf_24ss.registered_color)

--- Iterative Pairwise Object--- 
Algorithm used = colored_icp
Source file names : ['./datasets/4-multiway/24ss/24ss_positions_2_Detailed.csv', './datasets/4-multiway/24ss/24ss_positions_4_Detailed.csv', './datasets/4-multiway/24ss/24ss_positions_5_Detailed.csv']
Number of source files : 3
Target file name: ./datasets/4-multiway/24ss/24ss_positions_1_Detailed.csv
[[0.9461663947797716, 4.426941602180537, 4.67881931402874, 14.119038821162999], [0.9910536779324056, 3.991341263841081, 4.027371425701231, 14.164425143981822], [0.9497161394971614, 4.268511090435856, 4.494512531603254, 16.383053511181668]]
[[11.2545 ]
 [13.1143 ]
 [ 9.55597]
 ...
 [28.0678 ]
 [ 5.54132]
 [ 4.15403]]


# 27ss TCF-GFP

In [12]:
pos_path_source_list_27ss = ["./datasets/4-multiway/27ss/27ss_positions_2_Detailed.csv", 
                            "./datasets/4-multiway/27ss/27ss_positions_3_Detailed.csv", 
                            "./datasets/4-multiway/27ss/27ss_positions_4_Detailed.csv"]

color_path_source_list_27ss = ["./datasets/4-multiway/27ss/27ss_TCF_mRNA_2_Detailed.csv", 
                            "./datasets/4-multiway/27ss/27ss_TCF_mRNA_3_Detailed.csv", 
                            "./datasets/4-multiway/27ss/27ss_TCF_mRNA_4_Detailed.csv"]

pos_path_target_27ss = "./datasets/4-multiway/27ss/27ss_positions_1_Detailed.csv"
color_path_target_27ss = "./datasets/4-multiway/27ss/27ss_TCF_mRNA_1_Detailed.csv"

In [13]:
tcf_27ss = IterativePairwise(pos_path_source_list_27ss, pos_path_target_27ss, color_path_source_list_27ss, color_path_target_27ss,
                             algorithm = "colored_icp",pos_skiprows = [0,1,2], pos_usecols = [0,1,2], color_skiprows = [0,1,2], color_usecols = [0])

tcf_27ss.iterative_registration(jupyter_visualise = True, verbose = False)

Using median averaging
Using median averaging
Using median averaging


JVisualizer with 1 geometries

PointCloud with 1072 points.

## Results

In [14]:
print(tcf_27ss)
print(tcf_27ss.combined_results)
print(tcf_27ss.registered_color)

--- Iterative Pairwise Object--- 
Algorithm used = colored_icp
Source file names : ['./datasets/4-multiway/27ss/27ss_positions_2_Detailed.csv', './datasets/4-multiway/27ss/27ss_positions_3_Detailed.csv', './datasets/4-multiway/27ss/27ss_positions_4_Detailed.csv']
Number of source files : 3
Target file name: ./datasets/4-multiway/27ss/27ss_positions_1_Detailed.csv
[[0.8728897715988083, 4.405315432648464, 5.0468175661854415, 13.286058460043467], [0.8349650349650349, 4.7269094728480825, 5.661206487581874, 10.746549214754095], [0.8423326133909287, 4.738888921694707, 5.625911719858076, 13.61640235463426]]
[[4.0697 ]
 [3.18208]
 [2.44042]
 ...
 [3.24431]
 [3.5399 ]
 [2.70443]]


# 30ss TCF-GFP

In [15]:
pos_path_source_list_30ss = ["./datasets/4-multiway/30ss/30ss_positions_2_Detailed.csv", 
                            "./datasets/4-multiway/30ss/30ss_positions_3_Detailed.csv", 
                            "./datasets/4-multiway/30ss/30ss_positions_4_Detailed.csv"]

color_path_source_list_30ss = ["./datasets/4-multiway/30ss/30ss_TCF_mRNA_2_Detailed.csv", 
                            "./datasets/4-multiway/30ss/30ss_TCF_mRNA_3_Detailed.csv", 
                            "./datasets/4-multiway/30ss/30ss_TCF_mRNA_4_Detailed.csv"]

pos_path_target_30ss = "./datasets/4-multiway/30ss/30ss_positions_1_Detailed.csv"
color_path_target_30ss = "./datasets/4-multiway/30ss/30ss_TCF_mRNA_1_Detailed.csv"

In [16]:
tcf_30ss = IterativePairwise(pos_path_source_list_30ss, pos_path_target_30ss, color_path_source_list_30ss, color_path_target_30ss,
                             algorithm = "colored_icp",pos_skiprows = [0,1,2], pos_usecols = [0,1,2], color_skiprows = [0,1,2], color_usecols = [0])

tcf_30ss.iterative_registration(jupyter_visualise = True, verbose = False)

Using median averaging
Using median averaging
Using median averaging


JVisualizer with 1 geometries

PointCloud with 723 points.

## Results

In [17]:
print(tcf_30ss)
print(tcf_30ss.combined_results)
print(tcf_30ss.registered_color)

--- Iterative Pairwise Object--- 
Algorithm used = colored_icp
Source file names : ['./datasets/4-multiway/30ss/30ss_positions_2_Detailed.csv', './datasets/4-multiway/30ss/30ss_positions_3_Detailed.csv', './datasets/4-multiway/30ss/30ss_positions_4_Detailed.csv']
Number of source files : 3
Target file name: ./datasets/4-multiway/30ss/30ss_positions_1_Detailed.csv
[[0.9712140175219024, 4.205840509801781, 4.330498153777865, 13.25255238679289], [0.981838819523269, 4.522451523199233, 4.606103805709276, 14.062648630814083], [0.9267605633802817, 4.469329716623448, 4.822529025535939, 12.449271770604414]]
[[11.97485  ]
 [10.294    ]
 [12.047    ]
 [ 9.6051   ]
 [13.5733   ]
 [ 6.493    ]
 [15.1822   ]
 [14.0903   ]
 [12.8005   ]
 [12.95445  ]
 [13.576    ]
 [10.408    ]
 [ 8.29087  ]
 [27.0825   ]
 [ 6.62229  ]
 [17.3099   ]
 [14.8464   ]
 [15.3465   ]
 [ 8.08554  ]
 [11.5319   ]
 [ 9.89793  ]
 [12.0415   ]
 [ 2.8231   ]
 [ 9.317835 ]
 [ 4.85589  ]
 [10.312835 ]
 [ 7.04905  ]
 [ 5.87414  ]
 [ 